<a href="https://colab.research.google.com/github/donbcolab/vlm_architecture/blob/main/notebooks/mantis_8B_siglip_llama3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mantis 8B SigLIP Llama3 Vision

## Install Python Dependencies

In [1]:
# This only installs minimum packages (torch, transformers, accelerate) for inference, no redundant packages are installed.
!pip install git+https://github.com/TIGER-AI-Lab/Mantis.git

  Cloning https://github.com/TIGER-AI-Lab/Mantis.git to /tmp/pip-req-build-yge32g7a
  Running command git clone --filter=blob:none --quiet https://github.com/TIGER-AI-Lab/Mantis.git /tmp/pip-req-build-yge32g7a
  Resolved https://github.com/TIGER-AI-Lab/Mantis.git to commit e1ff2ffd524bc6e14477d8a1adda83ba1e29ac11
  Running command git submodule update --init --recursive -q
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 35.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (

## Import necessary libraries

In [2]:
from mantis.models.mllava import MLlavaProcessor, LlavaForConditionalGeneration
from mantis.models.mllava import chat_mllava
from PIL import Image
import torch

## Hugging Face model configuration
- Load the vision model and processor from Hugging Face

In [3]:
# Hugging Face model configuration
# note:  can support flash_attention_2

model_id = "TIGER-Lab/Mantis-8B-siglip-llama3"

# device = "cuda" if torch.cuda.is_available() else "cpu"

processor = MLlavaProcessor.from_pretrained(model_id)
attn_implementation = None # or "flash_attention_2"
model = LlavaForConditionalGeneration.from_pretrained(model_id, device_map="cuda", torch_dtype=torch.bfloat16, attn_implementation=attn_implementation)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/621 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.7k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/392 [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/2.16G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

## Visualize the Model Architecture

- Print the model architecture to understand its structure

In [4]:
# Visualize the Model Architecture

print(model)

LlavaForConditionalGeneration(
  (vision_tower): SiglipVisionModel(
    (vision_model): SiglipVisionTransformer(
      (embeddings): SiglipVisionEmbeddings(
        (patch_embedding): Conv2d(3, 1152, kernel_size=(14, 14), stride=(14, 14), padding=valid)
        (position_embedding): Embedding(729, 1152)
      )
      (encoder): SiglipEncoder(
        (layers): ModuleList(
          (0-26): 27 x SiglipEncoderLayer(
            (self_attn): SiglipAttention(
              (k_proj): Linear(in_features=1152, out_features=1152, bias=True)
              (v_proj): Linear(in_features=1152, out_features=1152, bias=True)
              (q_proj): Linear(in_features=1152, out_features=1152, bias=True)
              (out_proj): Linear(in_features=1152, out_features=1152, bias=True)
            )
            (layer_norm1): LayerNorm((1152,), eps=1e-06, elementwise_affine=True)
            (mlp): SiglipMLP(
              (activation_fn): PytorchGELUTanh()
              (fc1): Linear(in_features=1152, o

## Setup the input for the Vision model

## Setup and Run Vision model inference


## Discover the Model's Insights!